In [ ]:
# ⚙️ Install Required Packages
!pip install linearmodels statsmodels --quiet

# 📚 Import Libraries
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.formula.api import ols
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# 📁 Load Data
df = pd.read_csv("Environment_Regional_Dataset__Unbalanced_.csv")
df.dropna(inplace=True)
df['RegionID'] = df['RegionID'].astype('category')
df['Year'] = pd.to_numeric(df['Year'])

# ⚠️ Check Panel Balance
obs_counts = df.groupby('RegionID')['Year'].nunique()
print("📊 Unbalanced Panel Check:\n", obs_counts.describe())

# 🧱 Set Panel Index
df = df.set_index(['RegionID', 'Year'])

# 🎯 Define Regression Formula
dependent = 'PollutionIndex'
independent_vars = ['ForestCover', 'AvgTemperature', 'Rainfall']
formula = dependent + ' ~ ' + ' + '.join(independent_vars)

# 🔁 Fixed Effects Model
fe_model = PanelOLS.from_formula(formula + ' + EntityEffects', data=df).fit()
print("\n📌 Fixed Effects Summary:\n", fe_model)

# 🔁 Random Effects Model
re_model = RandomEffects.from_formula(formula, data=df).fit()
print("\n📌 Random Effects Summary:\n", re_model)

# 🧪 Hausman Test
def hausman(fe, re):
    b = fe.params[independent_vars]
    B = re.params[independent_vars]
    v_b = fe.cov.loc[independent_vars, independent_vars]
    v_B = re.cov.loc[independent_vars, independent_vars]
    stat = (b - B).T @ np.linalg.inv(v_b - v_B) @ (b - B)
    pval = 1 - stats.chi2.cdf(stat, len(b))
    return stat, pval

hausman_stat, p_val = hausman(fe_model, re_model)
print(f"\n🔍 Hausman Test Statistic: {hausman_stat:.3f}, p-value: {p_val:.3f}")
print("➡️ Use Fixed Effects" if p_val < 0.05 else "➡️ Use Random Effects")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00


<ipython-input-1-29f25bc9e1d9>:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  obs_counts = df.groupby('RegionID')['Year'].nunique()
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this wa

📊 Unbalanced Panel Check:
 count    60.000000
mean     10.200000
std       4.649768
min       5.000000
25%       7.000000
50%       8.000000
75%      14.000000
max      25.000000
Name: Year, dtype: float64

📌 Fixed Effects Summary:
                           PanelOLS Estimation Summary                           
Dep. Variable:         PollutionIndex   R-squared:                        0.0027
Estimator:                   PanelOLS   R-squared (Between):             -0.1025
No. Observations:                 612   R-squared (Within):               0.0027
Date:                Thu, May 29 2025   R-squared (Overall):             -0.0944
Time:                        20:35:09   Log-likelihood                   -2502.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.4979
Entities:                          60   P-value                           0.6839
Avg Obs:                       10.200 

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:601: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sum_weights: DataFrame = frame.groupby(level=level).transform("sum")
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:685: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weighted_sum = frame.groupby(level=level).sum()
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:687: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt 

In [ ]:
print("\n--- FIXED EFFECTS MODEL ---")
print(fe_model.summary)

print("\n--- RANDOM EFFECTS MODEL ---")
print(re_model.summary)

hausman_stat, p_val = hausman(fe_model, re_model)
print(f"\nHausman test: stat={hausman_stat:.3f}, p={p_val:.3f}")
print("➡️ Use Fixed Effects" if p_val < 0.05 else "➡️ Use Random Effects")


--- FIXED EFFECTS MODEL ---
                          PanelOLS Estimation Summary                           
Dep. Variable:         PollutionIndex   R-squared:                        0.0027
Estimator:                   PanelOLS   R-squared (Between):             -0.1025
No. Observations:                 612   R-squared (Within):               0.0027
Date:                Thu, May 29 2025   R-squared (Overall):             -0.0944
Time:                        20:35:09   Log-likelihood                   -2502.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.4979
Entities:                          60   P-value                           0.6839
Avg Obs:                       10.200   Distribution:                   F(3,549)
Min Obs:                       5.0000                                           
Max Obs:                       25.000   F-statistic (robust):             0.4979